# Example - Convert dataset to raster (GeoTiff)

Often, it is desirable to take a variable (band) out of your dataset and export it to a raster.
This is possible with the `rio.to_raster()` method. It does most of the work for you so you don't
have to.

Note: The `rio.to_raster()` method only works on a 2-dimensional or 3-dimensional `xarray.DataArray`

In [1]:
import rioxarray
import xarray

In [2]:
rds = xarray.open_dataset("../../test/test_data/input/PLANET_SCOPE_3D.nc")
rds

<xarray.Dataset>
Dimensions:      (time: 2, x: 10, y: 10)
Coordinates:
    spatial_ref  int64 ...
  * x            (x) float64 4.663e+05 4.663e+05 ... 4.663e+05 4.663e+05
  * time         (time) datetime64[ns] 2016-12-19T10:27:29.687763 2016-12-29T12:52:42.347451
  * y            (y) float64 8.085e+06 8.085e+06 ... 8.085e+06 8.085e+06
Data variables:
    blue         (time, y, x) float64 ...
    green        (time, y, x) float64 ...

In [3]:
rds.green.rio.to_raster?

Signature:
rds.green.rio.to_raster(
    raster_path,
    driver='GTiff',
    dtype=None,
    tags=None,
    windowed=False,
    **profile_kwargs,
)
Docstring:
Export the DataArray to a raster file.

Parameters
----------
raster_path: str
    The path to output the raster to.
driver: str, optional
    The name of the GDAL/rasterio driver to use to export the raster.
    Default is "GTiff".
dtype: str, optional
    The data type to write the raster to. Default is the datasets dtype.
tags: dict, optional
    A dictionary of tags to write to the raster.
windowed: bool, optional
    If True, it will write using the windows of the output raster.
    Default is False.
**profile_kwargs
    Additional keyword arguments to pass into writing the raster. The
    nodata, transform, crs, count, width, and height attributes
    are ignored.
File:      ~/scripts/rioxarray/rioxarray/rioxarray.py
Type:      method


In [4]:
rds.green.rio.to_raster("green_band.tif", tiled=True, compress="LZMA", windowed=True)

In [5]:
rds.green.shape, rds.green.rio.crs, rds.green.rio.nodata, rds.green.rio.bounds()

((2, 10, 10),
 CRS.from_epsg(32722),
 nan,
 (466266.0, 8084670.0, 466296.0, 8084700.0))

In [6]:
!rio info green_band.tif

{"blockxsize": 256, "blockysize": 256, "bounds": [466266.0, 8084670.0, 466296.0, 8084700.0], "colorinterp": ["gray", "undefined"], "compress": "lzma", "count": 2, "crs": "EPSG:32722", "descriptions": [null, null], "driver": "GTiff", "dtype": "float64", "height": 10, "indexes": [1, 2], "interleave": "pixel", "lnglat": [-51.31732641226951, -17.322997474192466], "mask_flags": [["nodata"], ["nodata"]], "nodata": NaN, "res": [3.0, 3.0], "shape": [10, 10], "tiled": true, "transform": [3.0, 0.0, 466266.0, 0.0, -3.0, 8084700.0, 0.0, 0.0, 1.0], "units": [null, null], "width": 10}


## Managing Information Loss with xarray operations

Sometimes, you can lose important information from your dataset when performing operations.
You will likely want to keep track of your `nodata` and your `CRS`.

In [7]:
new_ds = rds.green + rds.blue
new_ds.attrs, new_ds.rio.crs, new_ds.rio.nodata

(OrderedDict(), CRS.from_epsg(32722), None)

In [8]:
new_ds.rio.to_raster("combination.tif")

In [9]:
!rio info combination.tif

{"bounds": [466266.0, 8084670.0, 466296.0, 8084700.0], "colorinterp": ["gray", "undefined"], "count": 2, "crs": "EPSG:32722", "descriptions": [null, null], "driver": "GTiff", "dtype": "float64", "height": 10, "indexes": [1, 2], "interleave": "pixel", "lnglat": [-51.31732641226951, -17.322997474192466], "mask_flags": [["all_valid"], ["all_valid"]], "nodata": null, "res": [3.0, 3.0], "shape": [10, 10], "tiled": false, "transform": [3.0, 0.0, 466266.0, 0.0, -3.0, 8084700.0, 0.0, 0.0, 1.0], "units": [null, null], "width": 10}


In [10]:
new_ds = rds.green + rds.blue
new_ds.rio.write_crs(rds.green.rio.crs, inplace=True)
new_ds.rio.update_attrs(rds.green.attrs, inplace=True)
new_ds.attrs, new_ds.rio.crs, new_ds.rio.nodata

(OrderedDict([('grid_mapping', 'spatial_ref'),
              ('units', 'DN'),
              ('nodata', 0.0)]),
 CRS.from_epsg(32722),
 0.0)

In [11]:
new_ds.rio.to_raster("combination_keep_attrs.tif")

In [12]:
!rio info combination_keep_attrs.tif

{"bounds": [466266.0, 8084670.0, 466296.0, 8084700.0], "colorinterp": ["gray", "undefined"], "count": 2, "crs": "EPSG:32722", "descriptions": [null, null], "driver": "GTiff", "dtype": "float64", "height": 10, "indexes": [1, 2], "interleave": "pixel", "lnglat": [-51.31732641226951, -17.322997474192466], "mask_flags": [["nodata"], ["nodata"]], "nodata": 0.0, "res": [3.0, 3.0], "shape": [10, 10], "tiled": false, "transform": [3.0, 0.0, 466266.0, 0.0, -3.0, 8084700.0, 0.0, 0.0, 1.0], "units": [null, null], "width": 10}
